Setup Environment

In [15]:
#pipinstall vizdoom and clone repo in new folder
#!cd github & git clone https://github.com/Farama-Foundation/ViZDoom.git

#import vizdoom to setup game environment
from vizdoom import *
import random
import time
import numpy as np

In [17]:
actions = np.identity(3,dtype=np.uint8)
actions

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]], dtype=uint8)

In [18]:
random.choice(actions)

array([1, 0, 0], dtype=uint8)

In [5]:
game.new_episode()

In [6]:
game.is_episode_finished()

False

In [10]:
game.make_action(random.choice(actions))

-1.0

In [20]:
#number of games
episodes = 10

for episode in range(episodes):
    
    #new instance of game, similar to restarting
    game.new_episode()
    
    #while game is running
    while not game.is_episode_finished():
        
        #get game state
        state = game.get_state()
        #use game state to screen image
        img = state.screen_buffer
        #ise game state to grab game vars, i.e. ammo
        info = state.game_variables
        #take action, make_action() returns reward value for taking step
        reward = game.make_action(random.choice(actions),4) #2nd para is frame skip to give time between taking action and receiving result
        print("reward:", reward)
        time.sleep(.02)
    print("Result:", game.get_total_reward())
    time.sleep(2)
        
    

reward: -1.0
reward: -1.0
reward: -1.0
reward: -1.0
reward: -1.0
reward: -1.0
reward: -1.0
reward: -1.0
reward: -1.0
reward: -1.0
reward: 100.0
reward: -1.0
Result: 89.0
reward: -1.0
reward: -1.0
reward: -1.0
reward: -1.0
reward: -1.0
reward: -1.0
reward: 100.0
reward: -1.0
Result: 93.0


KeyboardInterrupt: 

In [21]:
game.close()

Setting up Doom with OpenAI Gym

In [24]:
#pip install and import gym environment
from gym import Env
from gym.spaces import Discrete, Box
import cv2

C:\Users\ahmed\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
actions[Discrete(3).sample()]

array([1, 0, 0], dtype=uint8)

In [34]:
Box(low=0,high=10,shape=(10,10),dtype=np.uint8).sample()

array([[ 1,  7,  1,  5,  7,  0,  8,  6, 10,  4],
       [ 4, 10,  4,  5,  9,  9,  3,  4,  4, 10],
       [ 8,  5,  1,  6,  4,  7,  0,  3, 10,  1],
       [ 0,  0,  9,  4,  7,  9, 10,  6,  7,  7],
       [ 4, 10,  0,  6,  2, 10,  1,  4,  8,  6],
       [ 4,  7,  4,  1,  8,  6,  1,  8,  6,  2],
       [ 0,  4,  3,  0,  7,  7,  1,  5,  5,  1],
       [ 4,  2,  4,  8,  2,  8,  7,  7,  8, 10],
       [ 7,  9,  8,  8,  0,  6,  5,  1,  1,  9],
       [ 0,  2,  8,  4,  2,  1,  0,  6,  7,  5]], dtype=uint8)

In [64]:
class VizDoomGym(Env):
    #called when env is started > game
    def __init__(self,render=False):
        self.game = DoomGame()
        self.game.load_config("github/VizDoom/scenarios/basic.cfg")
        
        #Disable or enable window visiblity when game is running
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        self.game.init()
    
        
        
        #create obs and action space
        #low/high indicates pixel vals
        self.observation_space = Box(low=0,high=255,shape=(3,320,240),dtype=np.uint8)
        self.action_space = Discrete(3)
    #tale actons
    def step(self,action):
        pass
    
    def render():
        pass
    
    #when game is restarted
    def reset():
        pass
    #grayscale frame and resize to make training faster
    def grayscale():
        pass
    #close the game
    def close(self):
        self.game.close()

In [69]:
env = VizDoomGym(render=True)

In [70]:
env

In [71]:
env.observation_space.sample().shape

(3, 320, 240)

In [72]:
env.close()